In [ ]:
import os
import numpy as np
from glob import glob
import xml.etree.ElementTree as ET

Find and parce the .xml files: main and MarkPoints. In this example picture they are the last and the first files, respectively. 

![alt text](image-1.png)

Using xml library, parce the file.

![alt text](image.png)

In [ ]:
data_dir = 'data_raw'
experimenter = 'calibration'
mouse = 'calibration_sb_mouse_fov0'
tseries = 'TSeries-10032023-1822-003'

In [ ]:
# Folder to find the .xml files:

folder_path = os.path.join(data_dir, experimenter, mouse, tseries)
#folder_path = '/Volumes/64g/ani73/References/'

folder_path = folder_path + '/'
tseries_main_xml_path = glob(folder_path + '*[0-9].xml')[0]
print(f'Found main .xml file: {tseries_main_xml_path}')
root = ET.parse(tseries_main_xml_path)
frame_times = [float(i.get('relativeTime')) for i in root.findall('.//Frame')]

mark_points_path = glob(folder_path + "*MarkPoints.xml")[0]
print(f'Found MarkPoints.xml file: {mark_points_path}')
root = ET.parse(mark_points_path)

stim_start_list = []
stim_end_list = []
points_list = []
coord_x_list = []
coord_y_list = []

time = 0

for series in root.findall('.'):
    iter_n = int(series.get('Iterations'))
    iter_delay = float(series.get('IterationDelay'))
    for iteration_i in range(iter_n):
        for mark_point_el in series.findall('./'):
            repetition_n = int(mark_point_el.get('Repetitions'))
            #Assuming only one PVGalvoPointElement in PVMarkPointElement
            init_delay = float(mark_point_el.find('./').get('InitialDelay'))
            time += init_delay
            for repetition_i in range(repetition_n):
                for galvo_point_el in mark_point_el.findall('./'):
                    inter_point_delay = float(galvo_point_el.get('InterPointDelay'))
                    duration = float(galvo_point_el.get('Duration'))
                    points = galvo_point_el.get('Points') 
                    
                    # Assuming there's only one Point per PVGalvoPointElement
                    point_x = float(galvo_point_el.find('./').get('X'))
                    point_y = float(galvo_point_el.find('./').get('Y'))

                    #stim_start_i = (iteration_i * iter_delay) + (repetition_i * inter_point_delay) + ((repetition_i) * duration) + overall_delay
                    #stim_end_i = (iteration_i * iter_delay) + (repetition_i * inter_point_delay) + ((repetition_i+1) * duration) + overall_delay

                    stim_start_i = time
                    time += duration
                    stim_end_i = time
                    
                    # Comparing with the frametime of the last recorded frame
                    if stim_start_i <= frame_times[-1]*1000:
                        stim_start_list.append(stim_start_i)
                        stim_end_list.append(stim_end_i)
                        points_list.append(points)
                        coord_x_list.append(point_x)
                        coord_y_list.append(point_y)
                        #overall_delay += stim_end_list[-1] - stim_start_list[-1]
                    else:
                        # If the recording was stopped earlier then all the planned stimulation, we break
                        break
                    
                    if repetition_i+1 != repetition_n:
                        time += inter_point_delay

        time += iter_delay

Making a dictionary that contains all the stimulation-related information.

In [ ]:
coord_x_list

In [ ]:
stim_dict = {'stim_points_name': points_list,
             'stim_coord_x': coord_x_list,
             'stim_coord_y': coord_y_list,
                'stim_start': stim_start_list,
                'stim_start_fr': np.searchsorted(frame_times, np.array(stim_start_list)/1000), # /1000 to convert ms to s
                'stim_end': stim_end_list,
                'stim_end_fr': np.searchsorted(frame_times, np.array(stim_end_list)/1000)
}

In [ ]:
print(stim_dict)

# TODO: Stim aritfact removal?

Here we select the folder that contains the results of suite2p analysis and extract the values. 
Written for single-plane recordings.

In [ ]:
import heapq

#s2p_folder_path = '/Volumes/64g/TSeries-02022024-1633-006/suite2p/plane0/'
#s2p_folder_path = '/Volumes/64g/ani73/suite2p/plane0/'
s2p_folder_path = '/Users/anaconda/Desktop/to_write_parcer/TSeries-02022024-1633-006/suite2p/plane0'

stat_npy = np.load(s2p_folder_path + '/stat.npy', allow_pickle=True)
iscell_npy = np.load(s2p_folder_path + '/iscell.npy')
ops_npy = np.load(s2p_folder_path + '/ops.npy', allow_pickle=True)[()]
F = np.load(s2p_folder_path + '/F.npy')

X_size = ops_npy['Lx']
Y_size = ops_npy['Ly']

X_all = []
Y_all = []

for i in range(stat_npy.shape[0]):
    X_all.append(np.mean(stat_npy[i]['xpix']))
    Y_all.append(np.mean(stat_npy[i]['ypix']))



Method to find the closest suite2p-detected cell for each pair of coordinates from the MarkPoints file.

In [ ]:
def find_closest_cells(stim_coord_x, stim_coord_y, X_all, Y_all, n_closest_cells = 5, iscell_only = False):

  def distance(x1, y1, x2, y2):
    return np.sqrt(((x1 - x2) ** 2) + ((y1 - y2) ** 2))

  if stat_npy.shape[0] > n_closest_cells:

    distances = []
    for j, (x, y) in enumerate(zip(X_all, Y_all)):
      dist = distance(stim_coord_x*X_size, stim_coord_y*Y_size, x, y)
      distances.append((dist, j))  # Store distance and index of cell

    #print(distances)

    closest_indices = []
    for dist, j in distances:
      heapq.heappush(closest_indices, (dist, j))
      
      # Get top 5 valid closest cells using nlargest
    
    if iscell_only:
      closest_cells = [(j, round(i,1)) for i, j in heapq.nsmallest(len(closest_indices), closest_indices, key=lambda x: x[0]) if iscell_npy[j][0] == 1][:n_closest_cells]
    else:
      closest_cells = [(j, round(i,1)) for i, j in heapq.nsmallest(n_closest_cells, closest_indices, key=lambda x: x[0])]
      
      return closest_cells
  else:
    print(f'You have less then {n_closest_cells+1} cell')

Finding the closest cells and adding tuples (closest cell number, distance) to the dictionary. The number of added tuples is defined by variable n_closest_cells = 5.


In [ ]:
closest_s2p_cells = []
for name, x, y in zip(stim_dict['stim_points_name'], stim_dict['stim_coord_x'], stim_dict['stim_coord_y']):
    print(name)
    print(find_closest_cells(x,y, X_all, Y_all))
    closest_s2p_cells.append(find_closest_cells(x,y, X_all, Y_all))

stim_dict['closest_s2p_cell(n_distance)'] = closest_s2p_cells

# TODO: write a convinient method to select the stimulated cell manually. Make an empty array for manual input. If it's empty, just use the closest (maybe also valid?) cell.
# Also, account for the cases where the time between the stimulatons is shorter then the timewindow




In [ ]:

from scipy.stats import ttest_rel
import matplotlib.pyplot as plt

# Stim cells:

# go throught each stim_start and stim_cell_n_i[0][0]
# redo methods to retrive the psth for all cells but single stim
# then do the tests and form the array that goes to the output table

# Isolating raw flourescence values that correspond to a specific stimulation frame
# Input: Stimulation frame number stim_fr_i, psth size (matematically: [-diapazone, diapazone]): diapazone
# Output: Flourescence traces for all cells before the stim ([stim_fr_i-diapazone, stim_fr_i]):psth_before, 
#         Flourescence traces for all cells after the stim ([stim_fr_i, stim_fr_i+diapazone]):psth_after 

def extract_psth(stim_fr_i, diapazone=30):
    psth_before = np.zeros((F.shape[0], diapazone))
    psth_after = np.zeros((F.shape[0], diapazone))
    for cell_i in range(F.shape[0]):
        psth_before[cell_i] = F[cell_i][stim_fr_i-diapazone:stim_fr_i]
        psth_after[cell_i] = F[cell_i][stim_fr_i:stim_fr_i+diapazone]

    return psth_before, psth_after

stim_cell_response_list = []

# Counting the number of values in an array (arr) that surpass the threshold (thr) CONSEQUTEVLY. Returns the maximum number of consequtive values.
def conseq_over_thr(arr, thr) -> int:
    if thr > 0:
        binarized = np.where(arr >= thr, 1, 0)
    else:
        binarized = np.where(arr <= thr, 1, 0)
    max_c_n = 0
    c_n = 0
    for i in binarized:
        if i == 1:
            c_n += 1
            if c_n > max_c_n:
                max_c_n = c_n
        else:
            c_n = 0
    return max_c_n


def plot_psth(array1, array2, text, diapazone):
    # Generate x values for each array
    #x_values1 = np.arange(-diapazone, 0)
    #x_values2 = np.arange(0, diapazone)
    x_val = np.arange(-diapazone, diapazone)
    y_val = [*array1, *array2]
    # Plot the data
    #plt.plot(x_values1, array1, label='Pre-stim')
    #plt.plot(x_values2, array2, label='Post-stim')
    plt.plot(x_val, y_val)
    plt.suptitle(text)
    plt.vlines(0, min(y_val) - 3, max(y_val) + 3, 'r', 'dashed')

    # Show the plot
    plt.show()


# Setting the cosecutive treshold and diapazone for 
conseq = 5
DIAPAZONE = 10

stim_indirect_response_pos = []
stim_indirect_response_neg = []

for stim_start_fr_i, stim_cell_n_i in zip(stim_dict['stim_start_fr'], stim_dict['closest_s2p_cell(n_distance)']):

    if stim_start_fr_i + DIAPAZONE < F.shape[1]: # Checking that there's enough frames after stim to make psth

        psth_before, psth_after = extract_psth(stim_start_fr_i, DIAPAZONE)
        stim_indirect_response_pos_i = []
        stim_indirect_response_neg_i = []
        for cell_i in range(F.shape[0]):
            if cell_i == stim_cell_n_i[0][0]:
                if ttest_rel(psth_before[cell_i], psth_after[cell_i])[1] <= 0.05:
                    stim_cell_response_list.append(1)
                else:
                    stim_cell_response_list.append(0)
                plot_psth(psth_before[cell_i], psth_after[cell_i], ttest_rel(psth_before[cell_i], psth_after[cell_i]), DIAPAZONE)

            else:
                if iscell_npy[cell_i][0] == 1:
                    baseAvg = np.mean(psth_before[cell_i])
                    baseStd = np.std(psth_before[cell_i])
                    #z_score_before = (psth_before - baseAvg) / baseStd
                    z_score_after = (psth_after[cell_i] - baseAvg) / baseStd
                    
                    if conseq_over_thr(z_score_after, 1.96) >= conseq:
                        stim_indirect_response_pos_i.append(cell_i)
                    if conseq_over_thr(z_score_after, -1.65) >= conseq:
                        stim_indirect_response_neg_i.append(cell_i)

    else:
        stim_cell_response_list.append(None)

    stim_indirect_response_pos.append(stim_indirect_response_pos_i)
    stim_indirect_response_neg.append(stim_indirect_response_neg_i)

In [ ]:
stim_dict['stim_cell_response'] = stim_cell_response_list
stim_dict['indir_pos_response'] = stim_indirect_response_pos
stim_dict['indir_neg_response'] = stim_indirect_response_neg

In [ ]:
import pandas as pd

pd.DataFrame.from_dict(stim_dict)

In [ ]:
# ChatGPT 3.5 code - neat but slower

from scipy.stats import ttest_rel
import matplotlib.pyplot as plt

def extract_psth(F, stim_fr_i, diapazone=30):
    psth_before = F[:, stim_fr_i - diapazone:stim_fr_i]
    psth_after = F[:, stim_fr_i + 1:stim_fr_i + diapazone + 1]
    return psth_before, psth_after

def conseq_over_thr(arr, thr):
    binarized = np.where((thr > 0) & (arr >= thr) | (thr <= 0) & (arr <= thr), 1, 0)
    max_c_n = 0
    c_n = 0
    for i in binarized:
        c_n = c_n + 1 if i == 1 else 0
        max_c_n = max(max_c_n, c_n)
    return max_c_n

conseq = 3
stim_cell_response_list = []
stim_indirect_response_pos = []
stim_indirect_response_neg = []

for stim_start_fr_i, stim_cell_n_i in zip(stim_dict['stim_start_fr'], stim_dict['closest_s2p_cell(n_distance)']):
    psth_before, psth_after = extract_psth(F, stim_start_fr_i, 30)
    stim_indirect_response_pos_i = [cell_i for cell_i in range(F.shape[0]) if cell_i != stim_cell_n_i[0][0] and conseq_over_thr((psth_after[cell_i] - np.mean(psth_before[cell_i])) / np.std(psth_before[cell_i]), 1.96) >= conseq]
    stim_indirect_response_neg_i = [cell_i for cell_i in range(F.shape[0]) if cell_i != stim_cell_n_i[0][0] and conseq_over_thr((psth_after[cell_i] - np.mean(psth_before[cell_i])) / np.std(psth_before[cell_i]), -1.65) >= conseq]

    stim_indirect_response_pos.extend(stim_indirect_response_pos_i)
    stim_indirect_response_neg.extend(stim_indirect_response_neg_i)

    if ttest_rel(psth_before[stim_cell_n_i[0][0]], psth_after[stim_cell_n_i[0][0]])[1] >= 0.90:
        stim_cell_response_list.append(1)
    else:
        stim_cell_response_list.append(0)


In [ ]:
# Sainty check - drawing the stimulated cells on the FOV shot.

from PIL import ImageDraw, Image, ImageFont

def extract_values(xml_data):
  """
  Extracts X, Y, and SpiralWidth values from a provided XML string.

  Args:
    xml_data: A string containing the XML data.

  Returns:
    A list of dictionaries, where each dictionary contains the extracted values
    for a single <Point> element.
  """

  root = ET.parse(xml_data)
  values = []

  for point in root.findall(".//Point"):
    values.append({
      "X": float(point.get("X")),
      "Y": float(point.get("Y")),
      "SpiralWidth": float(point.get("SpiralWidth")),
    })

  return values

extracted_values = extract_values('/Volumes/2TB_VIRUS/ani76_2023-02-12/TSeries-02122024-1600-002/References/TSeries-02122024-1600-002_Cycle00001_MarkPoints.xml')

print(extracted_values)


# Specify image path and circle diameter
image_path = "/Volumes/2TB_VIRUS/ani76_2023-02-12/TSeries-02122024-1600-002/Test.png"

# Open the image
img = Image.open(image_path)

# Create an ImageDraw object for drawing
draw = ImageDraw.Draw(img)

# Get image dimensions
width, height = img.size

def draw_circle_with_text(x, y, diameter, text=None):
    # Convert relative coordinates to absolute pixels
    center_x = int(x * width)
    center_y = int(y * height)
    radius = diameter // 2  # Calculate radius from diameter

    # Draw the circle with red color and 2px thickness
    draw.ellipse((center_x - radius, center_y - radius, center_x + radius, center_y + radius), outline=(255, 0, 0), width=2, fill='red')

    if text:
        # Load a font
        #font = ImageFont.truetype("arial.ttf", 5)  # Adjust font and size

        # Calculate text dimensions
        #text_width, text_height = 

        # Adjust text position relative to circle center
        text_x = center_x - 5 // 2
        text_y = center_y - 5 // 2

        # Draw the text in black color
        draw.text((text_x, text_y), text, fill=(255, 0, 0))


for n, cicle_dic in enumerate(extracted_values):
    # Example usage: Draw circles and text at specific relative coordinates
    draw_circle_with_text(cicle_dic['X'], cicle_dic['Y'], cicle_dic['SpiralWidth'], str(n))

# Save the modified image
img.save("/Volumes/2TB_VIRUS/ani76_2023-02-12/TSeries-02122024-1600-002/Test_with_c.png")


print("Image saved as modified_image.png")
